In [32]:
import numpy as np
import pandas as pd
import os, re, operator
from pprint import pprint

from src.utils import *
from src.k_nearest_neighbors import *
from src.logistic_regression import *
from src.naive_bayes import *
from src.neutral_network import *
#from src.support_vector_machines import *
from src.validation import *

#Base de dados
POSITIVE_TRUTHFUL  = ['op_spam_v1.4/positive/truthful/']
POSITIVE_DECEPTIVE = ['op_spam_v1.4/positive/deceptive/']
NEGATIVE_TRUTHFUL  = ['op_spam_v1.4/negative/truthful/']
NEGATIVE_DECEPTIVE = ['op_spam_v1.4/negative/deceptive/']
POSITIVE = [POSITIVE_TRUTHFUL, POSITIVE_DECEPTIVE]
NEGATIVE = [NEGATIVE_TRUTHFUL, NEGATIVE_DECEPTIVE]
DECEPTIVE = [POSITIVE_DECEPTIVE, NEGATIVE_DECEPTIVE]
TRUTHFUL = [POSITIVE_TRUTHFUL, NEGATIVE_TRUTHFUL]
ALL = [POSITIVE_TRUTHFUL, POSITIVE_DECEPTIVE,
           NEGATIVE_TRUTHFUL, NEGATIVE_DECEPTIVE]

#Parametros
    #Pre-processamento
USE_NGRAM = True
REMOVE_STOPWORD = True
REMOVE_ABOVE_THRESHOLD = 3
MIN_GRAM = 1
MAX_GRAM = 3
NORMALIZE = False
SAVE = False

    #SVM
COST = [i*0.3+0.1 for i in range(0, 100)]
GAMMA = [i*0.3+0.1 for i in range(0, 100)]
KERNEL = 2

    #FOLD
K_FOLD = 5

    #Outras constants
EMPTY = ''
ZERO = 0
ONE = 1
ENGLISH = 'english'
STOPWORDS = 'stopwords'

if K_FOLD == ZERO:
    raise Exception('Error on parameter for generate folds.')
if MIN_GRAM >= MAX_GRAM or MIN_GRAM < ONE or MAX_GRAM < ONE:
    raise Exception('Error on parameter for N-Gram.')

In [2]:
# Remoção de Stop Words
import nltk
from nltk.corpus import stopwords

# Download do dicionário de stop wordsgreat
nltk.download(STOPWORDS)

def stopwords_removal(tokens_list):
    '''
        Dado uma lista de tokens.
        Então é removido as stopwords.
        
        Entrada: tokens_list - Uma lista de tokens.
        Saida: new_tokens_list - Uma lista de tokens.
    '''
    if len(tokens_list) == ZERO:
        raise Exception('Error on stopwords_removal.')
        
    new_tokens_list = []
    
    stop_words = stopwords.words(ENGLISH)
    new_tokens_list = [token for token in tokens_list if token not in stop_words]
    
    if len(new_tokens_list) == ZERO:
        raise Exception('Error on stopwords_removal.')
        
    return new_tokens_list

[nltk_data] Downloading package stopwords to C:\Users\Vinícius F.
[nltk_data]     Carvalho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# N-gram dos tokens
def generate_ngram(tokens_list):
    '''
        Dado uma lista de tokens.
        Então é gerado N-Gram com base no MAX_GRAM e MIN_GRAM.
        
        Entrada: tokens_list - Uma lista de tokens.
        Saida: allNGrams - Uma lista de N-Gram.
    '''
    if len(tokens_list) == ZERO:
        raise Exception('Error on generate_ngram.')
        
    allNGrams = []
    
    #N-Gram
    for idx in range(MIN_GRAM, MAX_GRAM):
        ngrams = zip(*[tokens_list[i:] for i in range(idx)])
        allNGrams += ([" ".join(ngram) for ngram in ngrams])
    
    if len(allNGrams) == ZERO:
        raise Exception('Error on generate_ngram.')
        
    return allNGrams

In [4]:
# Remoção de features irrelevantes por threshold
def features_removal(features_dic, threshold):
    '''
        Dado um dicionario de features e um threshold.
        Então o dicionario é filtrado pelo numero do threshold.
        
        Entrada: features_dic - Um dicionario de features.
                 threshold - Um inteiro para limitar o minimo de ocorrencias de um item do dicionario.
        Saida: new_features_list - Um dicionario de features filtradas.
               new_features_dic - Um dicionario com as features filtradas.
    '''
    if len(features_dic) == ZERO or threshold < ONE:
        raise Exception('Error on features_removal.')

    new_features_dic = {}
    
    #Remove values < threshold
    for key, value in features_dic.items():
        if value > threshold:
            new_features_dic[key] = value

    new_features_list = [k for k, v in new_features_dic.items()]
    new_features_dic = sorted(new_features_dic.items(), key=operator.itemgetter(1))
    
    if len(new_features_list) == ZERO or len(new_features_dic) == ZERO:
        raise Exception('Error on features_removal.')
        
    return new_features_dic, new_features_list

In [33]:
# Tokenização dos textos
def pre_processing(paths, stopwords=False, ngram=False, threshold=0):
    '''
        Dado arquivos de textos.
        Então o texto é limpo (são removidos pontuação) e
            aplicado alguns métodos como remoção de stopwords, N-Gram e filtragem por threshold.
        
        Entrada: paths - Um dicionario de features.
                 stopwords - Um boolean indicando se é para utilizar este método.
                 ngram - Um boolean indicando se é para utilizar este método.
                 threshold - Um inteiro maior que zero que é utilizado na função features_removal.
        Saida: token_list - Uma lista contendo todas as features.
               token_dic - Um dicionario com as features e a sua ocorrencia.
    '''
    if len(paths) == ZERO:
        raise Exception('Error on pre_processing.')
        
    token_dic = {}
    dirs =[]
    token_list = []
    
    #Juntando arquivos de texto
    for p in paths:
        text_path = [p + t for t in os.listdir(p)]
        dirs += (text_path)
    
    for texts in dirs:
        with open(texts, 'r', encoding='utf-8') as stream:
            tokens = []
            text = stream.read()

            #Tokenizando o texto
            tokens = [word for word in (re.sub(r'[^\w\s]+','', text.replace('\n','')).lower().split(' ')) if word != '']
            
            #Tratando stopwords
            if stopwords:
                tokens = stopwords_removal(tokens)
            
            #Aplicando ngram
            if ngram:
                tokens = generate_ngram(tokens)
            
            #Criando dicionario de palavras
            for token in tokens:
                if token in token_dic:
                    token_dic[token] += 1
                else:
                    token_dic[token] = 1
    
    #Removendo palavras com base no threshold
    if threshold > ZERO:
        token_dic, token_list = features_removal(token_dic, threshold)
    
    if len(token_dic) == ZERO or len(token_list) == ZERO:
        raise Exception('Error on pre_processing.')
        
    return token_dic, token_list

# posTruthWordSplited, posTruthDic, listWords = wordsProcessed([POSITIVE_DECEPTIVE, NEGATIVE_DECEPTIVE], 7)
# posTruthWordSplited, posTruthDic, listWords = wordsProcessed([POSITIVE_DECEPTIVE], 3)
dicFeatures, listFeatures = pre_processing(NEGATIVE_DECEPTIVE, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD)
print('Pre-processamento finalizado.')
print('Foram gerados {} features.'.format(len(listFeatures)))

Pre-processamento finalizado.
Foram gerados 2069 features.


In [31]:
#Gera a matriz de features X e o vetor de classes Y
X = []
Y = []

def generateFeatures(paths, listFeatures, stopwords=False, ngram=False, threshold=0, example_class=0):
    '''
        Dado arquivos de textos.
        Então o dicionario é filtrado pelo numero do threshold
            e é gerado uma matriz que faz a contagem das features por texto.
        
        Entrada: paths - Textos a serem analizados para a geração das features.
                 listFeatures - Uma lista de features.
                 stopwords - Um boolean indicando se é para utilizar este método.
                 ngram - Um boolean indicando se é para utilizar este método.
                 threshold - Um inteiro maior que zero que é utilizado na função features_removal.
        Saida: Nenhuma.
    '''
    if len(paths) == ZERO:
        raise Exception('Error on generateFeatures.')
        
    dirs = []
    
    for p in paths:
        text_path = [p + t for t in os.listdir(p)]
        dirs += (text_path)
    
    for texts in dirs:
        row = np.zeros(len(listFeatures))
        with open(texts, 'r', encoding='utf-8') as stream:
            tokens = []
            text = stream.read()

            #Pre processing
            tokens = [word for word in (re.sub(r'[^\w\s]+','', text.replace('\n','')).lower().split(' ')) if word != '']
            
            if stopwords:
                tokens = stopwords_removal(tokens)
            
            if ngram:
                tokens = generate_ngram(tokens)
            
            #fix
            for token in tokens:
                for idx, feature in enumerate(listFeatures):
                    if token == feature:
                        row[idx] += 1
                        
            Y.append(example_class)
            X.append(row)
    
generateFeatures(NEGATIVE_DECEPTIVE, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 1)
generateFeatures(NEGATIVE_DECEPTIVE, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 0)
#generateFeatures(POSITIVE_DECEPTIVE, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 1)
#generateFeatures(POSITIVE_TRUTHFUL, listFeatures, REMOVE_STOPWORD, USE_NGRAM, REMOVE_ABOVE_THRESHOLD, 0)

#Transformando X e Y que são listas em array
X = np.array(X)
Y = np.array(Y)
print('Tratamento finalizado.')

Tratamento e pre-processamento finalizados.


In [34]:
#Salva os dados
if SAVE == True:
    save(np.column_stack((X, Y)), listFeatures, 'matrix_x')
    print('Dados salvos.')

In [35]:
#Normaliza os dados
X_norm = []
mu = 0
sigma = 0

if NORMALIZE == True:
    X_norm, mu, sigma = normalize(X)
    print('Dados normalizados.')

In [47]:
#Gera os folds necessários para os treinamentos  
folds = stratified_kfolds(Y, K_FOLD, np.unique(Y))

print("Foram gerados {} fold(s)".format(K_FOLD), end='\n\n')
idx = 0
for train_index, test_index in folds:
    print('Fold {} - Treino: {}'.format(idx, len(train_index)))
    print('Fold {} - Teste:  {}'.format(idx, len(test_index)), end='\n\n')
    idx += 1

Foram gerados 5 fold(s)

Fold 0 - Treino: 640
Fold 0 - Teste:  160

Fold 1 - Treino: 640
Fold 1 - Teste:  160

Fold 2 - Treino: 640
Fold 2 - Teste:  160

Fold 3 - Treino: 640
Fold 3 - Teste:  160

Fold 4 - Treino: 640
Fold 4 - Teste:  160



In [53]:
# Naive Bayes
def naiveBayes(folds, X, Y):
    '''
        Dado os folds, X e Y.
        Então é executado o algoritmo naive bayes para cada fold.
        
        Entrada: folds - Folds para serem testados.
                 X - Matriz de conjunto de dados.
                 Y - Lista com as classes correspondente aos dados da matriz X.
        Saida: resultados - Lista de resultados para cada fold.
               classes - Lista com as classes baseada na lista Y.
    '''
    resultados = []
    classes = np.unique(Y)

    for train_index, test_index in folds:
        # Treinamento
        probsPos, probsNeg = calcularProbabilidades(X[train_index], Y[train_index])

        # Classificação
        pred = []
        for x in [test_index]:
            probPos, probNeg = classificacao(x, probsPos, probsNeg, sum(Y == 1)/len(Y), sum(Y == 0)/len(Y))
            if (probPos >= probNeg):
                pred.append(1)
            else:
                pred.append(0)

        cm = get_confusionMatrix(Y[test_index], pred, np.unique(Y))
        resultado = relatorioDesempenho(cm, classes, False)

        resultados.append(resultado)
    
    return resultados, classes

resultados, classes = naiveBayes(folds, X, Y)
mediaFolds(resultados, classes)

IndexError: list index out of range

In [ ]:
# import random

# resultados = []
# for x in range(0, 5):
#     Y_resultado = np.array([random.randint(0, 1) for i in range(0, 160)])
#     Y_classes = np.array([random.randint(0, 1) for i in range(0, 160)])
#     classes = [0, 1]

#     cm = get_confusionMatrix(Y_classes, Y_resultado, classes)
    
    
#     resultado = relatorioDesempenho(cm, classes, False)
#     resultados.append(resultado)

# mediaFolds(resultados, classes)


# y = knn(X_val[81], X2, Y2, 5)
# #print(Y_val[81], ' ',y,' ', ind_viz)
# print(y)